In [ ]:
pip install tensorflow numpy matplotlib opencv-python


In [ ]:
# Licensed under the Creative Commons Attribution-NonCommercial 4.0 International License.

"""Main module for evaluating GAN-based anonymization techniques using various metrics."""

import dnnlib
from dnnlib import EasyDict
import dnnlib.tflib as tflib
import config
from metrics import metric_base
from training import misc

def evaluate_metric(submit_config, metric_args, network_pkl, dataset_args, mirror_augment):
    """Evaluate a given metric on a specified network pickle file for anonymization performance."""
    ctx = dnnlib.RunContext(submit_config)
    tflib.init_tf()
    print(f'Evaluating {metric_args.name} metric on anonymization network "{network_pkl}"...')
    metric = dnnlib.util.call_func_by_name(**metric_args)
    metric.run(network_pkl, dataset_args=dataset_args, mirror_augment=mirror_augment, num_gpus=submit_config.num_gpus)
    ctx.close()

def evaluate_snapshot(submit_config, metric_args, run_id, snapshot):
    """Evaluate a metric on a specific training snapshot of an anonymization model."""
    ctx = dnnlib.RunContext(submit_config)
    tflib.init_tf()
    print(f'Evaluating {metric_args.name} metric on run_id {run_id}, snapshot {snapshot}...')
    run_dir = misc.locate_run_dir(run_id)
    network_pkl = misc.locate_network_pkl(run_dir, snapshot)
    metric.run(network_pkl, run_dir=run_dir, num_gpus=submit_config.num_gpus)
    ctx.close()

def evaluate_all_snapshots(submit_config, metric_args, run_id):
    """Evaluate a metric across all snapshots of a specified run for an anonymization project."""
    ctx = dnnlib.RunContext(submit_config)
    tflib.init_tf()
    print(f'Evaluating {metric_args.name} metric on all snapshots of run_id {run_id} for anonymization performance...')
    run_dir = misc.locate_run_dir(run_id)
    network_pkls = misc.list_network_pkls(run_dir)
    for idx, network_pkl in enumerate(network_pkls):
        ctx.update('', idx, len(network_pkls))
        metric.run(network_pkl, run_dir=run_dir, num_gpus=submit_config.num_gpus)
    ctx.close()

def main():
    """Configure and submit tasks for evaluating metrics on GAN-based anonymization techniques."""
    submit_config = dnnlib.SubmitConfig()
    metrics = [metric_base.fid50k]  # Define which metrics to evaluate for anonymization effectiveness

    # Define the tasks for metric evaluation
    tasks = [
        EasyDict(
            run_func_name='run_metrics.evaluate_metric',
            network_pkl='https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ',
            dataset_args=EasyDict(tfrecord_dir='ffhq', shuffle_mb=0),
            mirror_augment=True
        )
    ]

    submit_config.num_gpus = 1  # Define number of GPUs to use
    submit_config.run_dir_root = dnnlib.submission.submit.get_template_from_path(config.result_dir)
    submit_config.run_dir_ignore += config.run_dir_ignore

    for task in tasks:
        for metric in metrics:
            submit_config.run_desc = f'{task.run_func_name}-{metric.name}-{"%dgpu" % submit_config.num_gpus}'
            dnnlib.submit_run(submit_config, metric_args=metric, **task)

if __name__ == "__main__":
    main()
